In [124]:
import pandas as pd
from pynif import NIFCollection
import requests
import time

In [16]:
s = ""
with open ('./training/oke17task1Training.xml.ttl') as f:
    s = f.read()

In [94]:
oke_task1_col = NIFCollection.loads(s, format='turtle')

for context in parsed_collection.contexts[:2]:
    print(context.mention)
    for phrase in context.phrases:
        print(f'\t{phrase}')

Prince Charles urged people to remember the story of the Nativity this Christmas, which was about "the fleeing of the Holy Family to escape violent persecution". He asked listeners to remember that the Prophet Mohammed migrated from Mecca to Medina because he was "seeking the freedom for himself and his followers to worship". The Prince said: "Whichever religious path we follow, the destination is the same - to value and respect the other person, accepting their right to live out their peaceful response to the love of God." 
	<NIFPhrase 202-218: 'Prophet Mohammed'>
	<NIFPhrase 233-238: 'Mecca'>
	<NIFPhrase 328-338: 'The Prince'>
	<NIFPhrase 242-248: 'Medina'>
	<NIFPhrase 0-14: 'Prince Charles'>
Derek Carr woke up a struggling Oakland offense by throwing two touchdown passes in the fourth quarter as the Raiders capped a successful trip to Mexico with a 27-20 victory over the Houston Texans on Monday night. Oakland had been held to 120 yards through three quarters, with the running game 

In [141]:
def get_scores(spots, ground_truth):
#     spots = sorted(spots, key=lambda s: s['offset'])
#     ground_truth = sorted(ground_truth, key=lambda s: s.beginIndex)
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    for spot in spots:
        matching = [x for x in ground_truth if x.beginIndex == spot['offset']]
        if not matching:
            false_positives += 1
            continue
        matching = matching[0]
        if spot['uri'] != matching.taIdentRef:
            false_positives += 1
        true_positives += 1
    for true_spot in ground_truth:
        matching = [x for x in spots if true_spot.beginIndex == x['offset']]
        if not matching:
            false_negatives += 1
#     print(f'tp: {true_positives} fp: {false_positives} fn: {false_negatives}')
    
    precision = 0
    recall = 0
    f1 = 0
    
    if true_positives + false_positives + false_negatives == 0:
        # Empty gold standard. Set all as 1
        precision = 1
        recall = 1
    
    if true_positives + false_positives != 0:
        precision = true_positives / (true_positives + false_positives)
    if true_positives + false_negatives != 0:
        recall = true_positives / (true_positives + false_negatives)
    if precision + recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    
    return {
        'tp': true_positives,
        'fp': false_positives,
        'fn': false_negatives,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [142]:
def _test_context(context, service_url, filter_policy):
    body = {
        'text': context.mention,
        'filter_policy': filter_policy
    }
    spots = requests.post(service_url, json=body).json()
    scores = get_scores(spots['data'], context.phrases)
    
    return scores

def test_collection(collection, service_url, filter_policy = 'NONE'):
    results = []
    for context in parsed_collection.contexts:
        result = _test_context(context, service_url, filter_policy)
        result['uri'] = context.uri
        results.append(result)
        time.sleep(0.5)
    return pd.DataFrame(results)

In [154]:
scores_spotlight = test_collection(oke_task1_col, 'http://localhost:4010/annotate')
scores_spotlight.head()

,tp,fp,fn,precision,recall,f1,uri
0,4,5,1,0.444444,0.800000,0.571429,http://www.ontologydesignpatterns.org/data/oke...
1,7,2,1,0.777778,0.875000,0.823529,http://www.ontologydesignpatterns.org/data/oke...
2,6,2,0,0.750000,1.000000,0.857143,http://www.ontologydesignpatterns.org/data/oke...
3,10,6,2,0.625000,0.833333,0.714286,http://www.ontologydesignpatterns.org/data/oke...
4,1,3,1,0.250000,0.500000,0.333333,http://www.ontologydesignpatterns.org/data/oke...


In [152]:
scores_fox = test_collection(oke_task1_col, 'http://localhost:4020/annotate')
scores_fox.head()

,tp,fp,fn,precision,recall,f1,uri
0,3,4,2,0.428571,0.600000,0.500000,http://www.ontologydesignpatterns.org/data/oke...
1,8,7,0,0.533333,1.000000,0.695652,http://www.ontologydesignpatterns.org/data/oke...
2,4,0,2,1.000000,0.666667,0.800000,http://www.ontologydesignpatterns.org/data/oke...
3,9,5,3,0.642857,0.750000,0.692308,http://www.ontologydesignpatterns.org/data/oke...
4,2,1,0,0.666667,1.000000,0.800000,http://www.ontologydesignpatterns.org/data/oke...


In [149]:
scores_tagme = test_collection(oke_task1_col, 'http://localhost:4030/annotate')
scores_tagme.head()

,tp,fp,fn,precision,recall,f1,uri
0,4,1,1,0.800000,0.800000,0.800000,http://www.ontologydesignpatterns.org/data/oke...
1,3,3,5,0.500000,0.375000,0.428571,http://www.ontologydesignpatterns.org/data/oke...
2,6,1,0,0.857143,1.000000,0.923077,http://www.ontologydesignpatterns.org/data/oke...
3,2,5,10,0.285714,0.166667,0.210526,http://www.ontologydesignpatterns.org/data/oke...
4,1,2,1,0.333333,0.500000,0.400000,http://www.ontologydesignpatterns.org/data/oke...


In [147]:
scores_all_none = test_collection(oke_task1_col, 'http://localhost:4040/annotate')
scores_all_none.head()

,tp,fp,fn,precision,recall,f1,uri
0,5,8,0,0.384615,1.000000,0.555556,http://www.ontologydesignpatterns.org/data/oke...
1,8,3,0,0.727273,1.000000,0.842105,http://www.ontologydesignpatterns.org/data/oke...
2,6,0,0,1.000000,1.000000,1.000000,http://www.ontologydesignpatterns.org/data/oke...
3,10,7,2,0.588235,0.833333,0.689655,http://www.ontologydesignpatterns.org/data/oke...
4,2,4,0,0.333333,1.000000,0.500000,http://www.ontologydesignpatterns.org/data/oke...


In [144]:
scores_all_r_singles = test_collection(oke_task1_col, 'http://localhost:4050/annotate', filter_policy='REMOVE_SINGLE_SPOTS')
scores_all_r_singles.head()

,tp,fp,fn,precision,recall,f1,uri
0,4,1,1,0.800000,0.800,0.800000,http://www.ontologydesignpatterns.org/data/oke...
1,7,0,1,1.000000,0.875,0.933333,http://www.ontologydesignpatterns.org/data/oke...
2,6,2,0,0.750000,1.000,0.857143,http://www.ontologydesignpatterns.org/data/oke...
3,9,5,3,0.642857,0.750,0.692308,http://www.ontologydesignpatterns.org/data/oke...
4,1,2,1,0.333333,0.500,0.400000,http://www.ontologydesignpatterns.org/data/oke...


In [156]:
scores_spotlight.to_csv('scores/scores_spotlight_oke1.csv', index=False)
scores_fox.to_csv('scores/scores_fox_oke1.csv', index=False)
scores_tagme.to_csv('scores/scores_tagme_oke1.csv', index=False)

scores_all_none.to_csv('scores/scores_all_none_oke1.csv', index=False)
scores_all_r_singles.to_csv('scores/scores_all_r_singles_oke1.csv', index=False)